In [ ]:
import kfp
from kfp.v2.dsl import component, pipeline
from kfp.v2 import compiler
from kfp import dsl


@dsl.component (base_image='python:3.8',  # Specify the base image
    packages_to_install=['pandas'])
def load_and_print_csv(bucket_name: str, file_name: str, minio_endpoint: str, access_key: str, secret_key: str,local_dir: str):
    import subprocess
    import pandas as pd

    # Install MinIO client
    subprocess.run(['apt-get', 'update'], check=True)
    subprocess.run(['apt-get', 'install', '-y', 'wget'], check=True)
    #subprocess.run(['apt-get', 'install', '-y', 'iputils-ping'], check=True)
    #subprocess.run(['apt-get', 'install', '-y', 'coreutils'], check=True)
    #subprocess.run(['timeout','4','ping','minio-part2.partdp.ir'], check=True)
    subprocess.run(['wget', 'https://dl.min.io/client/mc/release/linux-amd64/mc'], check=True)
    subprocess.run(['chmod', '+x', 'mc'], check=True)
    

    # Set MinIO alias
    print(f"Setting MinIO alias with endpoint {minio_endpoint}")
    subprocess.run(['./mc', 'alias', 'set', 'mlopsminio', minio_endpoint, access_key, secret_key,'--insecure'], check=True)
    #subprocess.run(['./mc','alias','ls'], check=True)
    #subprocess.run(['./mc','ls','mlopsminio','--insecure'], check=True)

    
    
    # Download CSV file from MinIO bucket
    print(f"Downloading {file_name} from bucket {bucket_name}")
    subprocess.run(['./mc', 'cp', f'mlopsminio/{bucket_name}/{file_name}', 'test26355342.csv','--insecure'], check=True)

    # Read and print the CSV content using pandas
    print(f"Reading CSV file {file_name}")
    data = pd.read_csv('test26355342.csv')
    print(data)

    # Upload CSV file from MinIO bucket
    subprocess.run(['./mc', 'cp', '--recursive','test26355342.csv', f'mlopsminio/{bucket_name}/','--insecure'], check=True)

    

# Define the pipeline using the Kubeflow v2 DSL
@dsl.pipeline(
    name='Load and Print CSV Pipeline',
    description='A pipeline to load a CSV file from MinIO and print its data'
)
def pipeline():
    minio_endpoint = 'https://minio-part2.partdp.ir:9000'  # MinIO service in Kubeflow
    local_dir="."
    bucket_name= "mlops"
    file_name = "test.csv"
    access_key="IKdBTTGbegWP8KVnRZei"
    secret_key="1G0ijK1DBBAEYMXJW7j5ep97OuwMDg44h7HzdSbX"
    
    # Call the component that downloads and prints the CSV file
    task1 = load_and_print_csv(
        bucket_name=bucket_name,
        file_name=file_name,
        minio_endpoint=minio_endpoint,
        access_key=access_key,
        secret_key=secret_key,
        local_dir=local_dir
    )
    task1.set_cpu_limit("4").set_memory_limit("4G").set_gpu_limit("2").set_caching_options(False)

 client = kfp.Client()
client.create_run_from_pipeline_func(
    pipeline,
    arguments={},
    experiment_name="Default",
    enable_caching=False
)